> Trabalho da disciplina de Inteligência Artificial sobre Algoritmos de Aprendizado de Máquina.
>> Feito por: **Henrique Navarro Morais.**

# Importações necessárias

In [1]:
import pandas as pd
import random
import numpy as np
from sklearn import *

In [ ]:
#Instação da extensão 'Collapsible Headings' para melhor visualização dos cabeçalhos
import os
os.system('pip install -q jupyter_contrib_nbextensions')
os.system('jupyter contrib nbextension install --user')
os.system('jupyter nbextension enable collapsible_headings/main --quiet')

# Sobre a Base de Dados

> A base de dados possui diversos atributos sobre diversas pessoas adultas dos EUS, e seu principal objetivo é apontar se a renda do indivíduo é inferior ou superior a 50K.
>
> Link da base de dados: https://archive.ics.uci.edu/ml/datasets/Adult

 > Foram feitas diversas operações na base de dados original, de forma que os dados foram tratados, limpos, convertidos e padronizados. Justamante para melhorar a qualidade e a confiabilidade dos dados a fim de serem usados por algoritmos de aprendizado de máquina.
> 
> Será feita a importação de 2 versões da base de dados, uma versão sem os atributos desnecessários, sem os exemplos denecessários e duplicados, e outra com essas mesmas características, porém, com todos os dados convertidos para numéricos e padronizados.
>
> O atributo de saída da base de dados é a **Renda**, de forma que na base de dados numérica, padronizada com valores de -10 a +10, os exemplos com valor igual a <=50K serão representadas como -10, já os exemplos que assumirem o valor de >50K, serão representados como +10.

In [2]:
db_clean = pd.read_csv('./databases/db_clean.csv')
db = pd.read_csv('./databases/db_num_ree.csv')

In [3]:
display(db_clean.head())
display(db.head())

,Idade,Classe Trabalhadora,Educacao,Num Educacao,Ocupacao,Ganho de Capital,Perda de Capital,Horas por Semana,Renda
0,39,State-gov,Bachelors,13,Adm-clerical,2174,0,40,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Exec-managerial,0,0,13,<=50K
2,38,Private,HS-grad,9,Handlers-cleaners,0,0,40,<=50K
3,53,Private,11th,7,Handlers-cleaners,0,0,40,<=50K
4,28,Private,Bachelors,13,Prof-specialty,0,0,40,<=50K


,Idade,Classe Trabalhadora,Educacao,Num Educacao,Ocupacao,Ganho de Capital,Perda de Capital,Horas por Semana,Renda
0,-3.230769,-10.0,-10.000000,6.000000,-10.000000,-8.277065,-10.0,-0.370370,-10.0
1,0.153846,-7.5,-10.000000,6.000000,-8.571429,-10.000000,-10.0,-7.037037,-10.0
2,-3.538462,-5.0,-8.666667,0.666667,-7.142857,-10.000000,-10.0,-0.370370,-10.0
3,1.076923,-5.0,-7.333333,-2.000000,-7.142857,-10.000000,-10.0,-0.370370,-10.0
4,-6.615385,-5.0,-10.000000,6.000000,-5.714286,-10.000000,-10.0,-0.370370,-10.0


# Técnica de Validação

> O método hold-out envolve a divisão do conjunto de dados em dois subconjuntos distintos: um conjunto de **treinamento** e um conjunto de **teste.**
> 
> A base de dados analisada foi dividida **20%** dos exemplos serão usados como **testes** e os **80%** restantes dos exemplos serão usados para **treino.**

In [5]:
def split_dataset(dataset, test_ratio):
    dataset_array = dataset.to_numpy()
    num_samples = len(dataset_array)
    np.random.shuffle(dataset_array)
    num_test_samples = int(num_samples * test_ratio)
    train_data = dataset_array[num_test_samples:]
    test_data = dataset_array[:num_test_samples]
    train_data = pd.DataFrame(train_data, columns=dataset.columns)
    test_data = pd.DataFrame(test_data, columns=dataset.columns)
    
    return train_data, test_data

db_treino, db_teste = split_dataset(db, test_ratio=0.2)
#db_clean_treino, db_clean_teste = split_dataset(db_clean, test_ratio=0.2)

In [6]:
print(f'Tamanho subconjunto de treino: {db_treino.shape[0]} exemplos\nTamanho subconjunto de teste: {db_teste.shape[0]} exemplos')
display(db_treino.head(3))
display(db_teste.head(3))

Tamanho subconjunto de treino: 17920 exemplos
Tamanho subconjunto de teste: 4480 exemplos


,Idade,Classe Trabalhadora,Educacao,Num Educacao,Ocupacao,Ganho de Capital,Perda de Capital,Horas por Semana,Renda
0,2.923077,-5.0,-3.333333,2.000000,-8.571429,-10.0,-1.515152,0.864198,10.0
1,1.076923,-5.0,-8.666667,0.666667,-4.285714,-10.0,-10.000000,-0.370370,-10.0
2,-6.615385,-5.0,-8.666667,0.666667,0.000000,-10.0,-2.011019,-0.370370,-10.0


,Idade,Classe Trabalhadora,Educacao,Num Educacao,Ocupacao,Ganho de Capital,Perda de Capital,Horas por Semana,Renda
0,-6.923077,-2.5,-10.000000,6.000000,7.142857,-10.0,-10.0,-1.604938,-10.0
1,1.692308,-5.0,-7.333333,-2.000000,2.857143,-10.0,-10.0,-2.345679,-10.0
2,-6.000000,-7.5,-0.666667,3.333333,-1.428571,-10.0,-10.0,-0.370370,-10.0


# Métricas

> Converter dados não numéricos em uma representação numérica é uma etapa crucial para garantir que os algoritmos possam processar e extrair insights úteis dos dados.

In [7]:
def convert_labels_to_numeric(labels):
    unique_labels = list(set(labels))
    label_map = {label: i for i, label in enumerate(unique_labels)}
    numeric_labels = [label_map[label] for label in labels]
    return numeric_labels

> **Acurácia:** é uma medida que indica a taxa de classificações corretas em relação ao total de amostras. É uma medida geral de desempenho do modelo, indicando a proporção de previsões corretas em relação ao número total de previsões.

In [8]:
def calculate_accuracy(y_true, y_pred):
    correct_predictions = 0
    total_predictions = len(y_true)

    for true_label, predicted_label in zip(y_true, y_pred):
        if true_label == predicted_label:
            correct_predictions += 1

    accuracy = correct_predictions / total_predictions

    return accuracy

> **Precisão:** é uma medida que indica a proporção de verdadeiros positivos em relação ao total de positivos previstos (verdadeiros positivos + falsos positivos).

In [9]:
def calculate_precision(y_true, y_pred):
    true_positives = 0
    false_positives = 0

    for true_label, predicted_label in zip(y_true, y_pred):
        if predicted_label == 1:
            if true_label == predicted_label:
                true_positives += 1
            else:
                false_positives += 1

    if true_positives + false_positives == 0:
        precision = 0
    else:
        precision = true_positives / (true_positives + false_positives)

    return precision

> **Recall:** também conhecido como taxa de verdadeiros positivos ou sensibilidade, é uma medida que indica a proporção de verdadeiros positivos em relação ao total de positivos reais (verdadeiros positivos + falsos negativos).

In [10]:
def calculate_recall(y_true, y_pred):
    true_positives = 0
    false_negatives = 0

    for true_label, predicted_label in zip(y_true, y_pred):
        if true_label == predicted_label == 1:
            true_positives += 1
        elif true_label == 1 and predicted_label == 0:
            false_negatives += 1

    if true_positives + false_negatives == 0:
        recall = 0
    else:
        recall = true_positives / (true_positives + false_negatives)

    return recall

> **Matriz de confusão:** uma tabela que mostra o desempenho do modelo de classificação em relação às classes reais. Ela apresenta a contagem de verdadeiros positivos, falsos positivos, verdadeiros negativos e falsos negativos.

In [11]:
def calculate_confusion_matrix(y_true, y_pred):
    true_positives = 0
    true_negatives = 0
    false_positives = 0
    false_negatives = 0

    for true_label, predicted_label in zip(y_true, y_pred):
        if true_label == predicted_label == 1:
            true_positives += 1
        elif true_label == predicted_label == 0:
            true_negatives += 1
        elif true_label == 0 and predicted_label == 1:
            false_positives += 1
        elif true_label == 1 and predicted_label == 0:
            false_negatives += 1

    confusion = [[true_negatives, false_positives], [false_negatives, true_positives]]

    return confusion

# Algoritmo Baseline

> **Algoritmo baseline** é um método simples que serve como ponto de referência para avaliar o desempenho de modelos mais avançados. Nesse caso, foi utilizado um algoritmo que calcula a frequência de cada classe no conjunto de treinamento e, em seguida, atribui a classe majoritária a todas as instâncias do conjunto de teste.

In [29]:
class_counts = db_treino['Renda'].value_counts()
majority_class = class_counts.idxmax()
majority_class_proportion = class_counts[majority_class] / len(db_treino)

def baseline_predict():
    return np.repeat(majority_class, len(db_teste))

baseline_predictions = baseline(db_treino, db_teste, 'Renda')
print(f'{baseline_predictions}')


[-10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0

In [14]:
# Calcula a frequência de cada classe no conjunto de treinamento
def baseline(train_data, test_data, target_column):
    class_counts = {}
    for _, row in train_data.iterrows():
        class_label = row[target_column]
        if class_label in class_counts:
            class_counts[class_label] += 1
        else:
            class_counts[class_label] = 1

    majority_class = max(class_counts, key=class_counts.get)
    majority_class_count = class_counts[majority_class]

    predictions = [majority_class] * len(test_data)

    return majority_class, majority_class_count

majority_class, majority_class_count = baseline(db_treino, db_teste, 'Renda')
print(f'A classe mais frequente na base de dados é: {majority_class} e possui {majority_class_count} exemplos')

A classe mais frequente na base de dados é: -10.0 e possui 12735 exemplos


> Como a base de dados foi padronizada com valores entre o intervalo numérico de **[-10 a +10]**, a classe mais frequente na base de dados é **-10**, ou seja, que representa justamente o valor **<=50K** assumido pelo atributo **Renda.**

# K-NN

> O algoritmo **K-NN** atribui rótulos às instâncias de teste com base nos rótulos das instâncias vizinhas mais próximas no espaço de atributos. O valor de **k** determina o **número de vizinhos** considerados.

In [36]:
from sklearn.neighbors import KNeighborsClassifier
def knn_model(train_data, test_data, k):
    train_data = train_data.copy()
    test_data = test_data.copy()

    X_train = train_data.drop('Renda', axis=1)
    y_train = train_data['Renda']
    X_test = test_data.drop('Renda', axis=1)
    y_test = test_data['Renda']

    for column in X_train.columns:
        if X_train[column].dtype != 'object':
            X_train[column] = pd.cut(X_train[column], bins=5, labels=False)
            X_test[column] = pd.cut(X_test[column], bins=5, labels=False)

    y_train = convert_labels_to_numeric(y_train)
    y_test = convert_labels_to_numeric(y_test)

    knn = KNeighborsClassifier(n_neighbors=k)

    knn.fit(X_train, y_train)

    y_pred = knn.predict(X_test)

    accuracy = calculate_accuracy(y_test, y_pred)
    precision = calculate_precision(y_test, y_pred)
    recall = calculate_recall(y_test, y_pred)
    confusion = calculate_confusion_matrix(y_test, y_pred)

    return accuracy, precision, recall, confusion

In [35]:
k_values = [1, 5, 20, 50, 100]
results = []

for k in k_values:
    accuracy, precision, recall, confusion = knn_model(db_treino, db_teste, k)
    results.append([k, accuracy, precision, recall, confusion])

columns = ['k', 'accuracy', 'precision', 'recall', 'confusion']
metricas_res = pd.DataFrame(results, columns=columns)
display(metricas_res)
#display(confusao)

,k,accuracy,precision,recall,confusion
0,1,0.719420,0.801014,0.800760,"[[695, 628], [629, 2528]]"
1,5,0.770089,0.806572,0.886284,"[[652, 671], [359, 2798]]"
2,20,0.785268,0.804438,0.918594,"[[618, 705], [257, 2900]]"
3,50,0.785491,0.796436,0.934431,"[[569, 754], [207, 2950]]"
4,100,0.786161,0.788356,0.952170,"[[516, 807], [151, 3006]]"


> É possível verificar médidas do desempenho do algoritmo K-NN com diversas variações de **K**

# Árvore C4.5

> O algoritmo **C4.5** é um algoritmo de árvore de decisão que constrói uma árvore de decisão onde cada nó interno representa um teste em um atributo e cada ramo representa um resultado desse teste. 
> 
> O **C4.5** utiliza o ganho de informação para determinar a melhor divisão dos dados em cada nó da árvore.

In [31]:
from sklearn.tree import DecisionTreeClassifier
def arvorec4_5(train_data, test_data, criterio):
    train_data = train_data.copy()
    test_data = test_data.copy()

    X_train = train_data.drop('Renda', axis=1)
    y_train = train_data['Renda']
    X_test = test_data.drop('Renda', axis=1)
    y_test = test_data['Renda']

    for column in X_train.columns:
        if X_train[column].dtype != 'object':
            X_train[column] = pd.cut(X_train[column], bins=5, labels=False)
            X_test[column] = pd.cut(X_test[column], bins=5, labels=False)

    y_train = convert_labels_to_numeric(y_train)
    y_test = convert_labels_to_numeric(y_test)

    c4_5 = DecisionTreeClassifier(criterion=criterio)
    c4_5.fit(X_train, y_train)

    y_pred = c4_5.predict(X_test)

    accuracy = calculate_accuracy(y_test, y_pred)
    precision = calculate_precision(y_test, y_pred)
    recall = calculate_recall(y_test, y_pred)
    confusion = calculate_confusion_matrix(y_test, y_pred)

    return accuracy, precision, recall, confusion

In [37]:
results = []
metricas = ['entropy', 'gini', 'friedman_mse', 'mse']

for metrica in metricas:
    accuracy, precision, recall, confusion = arvorec4_5(db_treino, db_teste, 'entropy')
    results.append([metrica, accuracy, precision, recall, confusion])
    
columns = ['criterio', 'accuracy', 'precision', 'recall', 'confusion']

metricas_res = pd.DataFrame(results, columns=columns)
display(metricas_res)
#display(confusao)

,criterio,accuracy,precision,recall,confusion
0,entropy,0.785937,0.812039,0.905923,"[[661, 662], [297, 2860]]"
1,gini,0.786384,0.812323,0.906240,"[[662, 661], [296, 2861]]"
2,friedman_mse,0.785714,0.811985,0.905607,"[[661, 662], [298, 2859]]"
3,mse,0.786607,0.812376,0.906557,"[[662, 661], [295, 2862]]"


> Possível verificar o desempenho do modelo com base em diferentes métricas

# MLP

> **MLP**  é uma arquitetura de rede neural artificial composta por múltiplas camadas de neurônios. Cada neurônio em uma camada está conectado a todos os neurônios nas camadas adjacentes. 
> 
> O **MLP** é treinado usando o algoritmo de backpropagation, que ajusta os pesos das conexões para minimizar a diferença entre as saídas previstas e os valores de destino.

> retirar bibliotecas

In [27]:
import warnings
from sklearn.exceptions import ConvergenceWarning
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

def mlp_model(train_data, test_data, hidden_layer_sizes=(50,), activation='relu', solver='adam', random_state=42):
    train_data = train_data.copy()
    test_data = test_data.copy()

    X_train = train_data.drop('Renda', axis=1)
    y_train = train_data['Renda']
    X_test = test_data.drop('Renda', axis=1)
    y_test = test_data['Renda']

    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=ConvergenceWarning)
        
        mlp = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes, activation=activation, solver=solver, max_iter=200, random_state=random_state)
        mlp.fit(X_train, y_train)

        y_pred = mlp.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted', zero_division=1)
    recall = recall_score(y_test, y_pred, average='weighted', zero_division=1)
    confusion = confusion_matrix(y_test, y_pred)

    return accuracy, precision, recall, confusion

results = []
hidden_layer_sizes = [(50,), (25,), (100,)]
activations = ['relu', 'logistic', 'tanh']
solvers = ['adam', 'sgd']
random_state = 42

for hidden_layers in hidden_layer_sizes:
    for activation in activations:
        for solver in solvers:
            accuracy, precision, recall, confusion = mlp_model(db_treino, db_teste, hidden_layer_sizes=hidden_layers, activation=activation, solver=solver, random_state=random_state)
            results.append([hidden_layers, activation, solver, accuracy, precision, recall, confusion])

columns = ['hidden_layers', 'activation', 'solver', 'accuracy', 'precision', 'recall', 'confusion']
metricas_res = pd.DataFrame(results, columns=columns)
display(metricas_res)

,hidden_layers,activation,solver,accuracy,precision,recall,confusion
0,"(50,)",relu,adam,0.787723,0.777778,0.787723,"[[2879, 285], [666, 650]]"
1,"(50,)",relu,sgd,0.781920,0.771210,0.781920,"[[2902, 262], [715, 601]]"
2,"(50,)",logistic,adam,0.795089,0.788400,0.795089,"[[2960, 204], [714, 602]]"
3,"(50,)",logistic,sgd,0.785714,0.779893,0.785714,"[[2985, 179], [781, 535]]"
4,"(50,)",tanh,adam,0.791741,0.784428,0.791741,"[[2956, 208], [725, 591]]"
5,"(50,)",tanh,sgd,0.786607,0.778177,0.786607,"[[2948, 216], [740, 576]]"
6,"(25,)",relu,adam,0.787500,0.782491,0.787500,"[[2991, 173], [779, 537]]"
7,"(25,)",relu,sgd,0.787277,0.779998,0.787277,"[[2965, 199], [754, 562]]"
8,"(25,)",logistic,adam,0.790848,0.784083,0.790848,"[[2966, 198], [739, 577]]"
9,"(25,)",logistic,sgd,0.785714,0.779170,0.785714,"[[2977, 187], [773, 543]]"


# Resultados Baseline

In [73]:
predictions = baseline_predict()

accuracy = (predictions == db_teste['Renda']).mean()

confusion_matrix = pd.crosstab(db_teste['Renda'], predictions, rownames=['Real'], colnames=['Predito'], dropna=False)

true_positive = confusion_matrix[majority_class][majority_class]
false_positive = confusion_matrix[majority_class].sum() - true_positive
false_negative = confusion_matrix.drop(majority_class, axis=1).sum().sum()

precision = true_positive / (true_positive + false_positive)
recall = true_positive / (true_positive + false_negative)

# Imprimir os resultados
print("Acurácia:", accuracy)
print("Precisão:", precision)
print("Recall:", recall)
print("Matriz de Confusão:")
print(confusion_matrix)


Acurácia: 0.7118303571428571
Precisão: 0.7118303571428571
Recall: 1.0
Matriz de Confusão:
Predito  -10.0
Real          
-10.0     3189
 10.0     1291


In [69]:
def calculate_accuracy_baseline(y_true, majority_class):
    correct_predictions = y_true.count(majority_class)
    total_predictions = len(y_true)
    accuracy = correct_predictions / total_predictions
    return accuracy

def calculate_precision_baseline(y_true, majority_class):
    true_positives = y_true.count(majority_class)
    total_predictions = len(y_true)
    false_positives = total_predictions - true_positives
    if true_positives + false_positives == 0:
        precision = 0
    else:
        precision = true_positives / (true_positives + false_positives)
    return precision

def calculate_recall_baseline(y_true, majority_class):
    true_positives = y_true.count(majority_class)
    false_negatives = len(y_true) - true_positives
    if true_positives + false_negatives == 0:
        recall = 0
    else:
        recall = true_positives / (true_positives + false_negatives)
    return recall

def calculate_confusion_matrix_baseline(y_true, majority_class):
    true_positives = y_true.count(majority_class)
    true_negatives = 0
    false_positives = 0
    false_negatives = len(y_true) - true_positives
    confusion_matrix = [[true_negatives, false_positives], [false_negatives, true_positives]]
    return confusion_matrix


In [57]:
def calculate_metrics_baseline():
    accuracy = calculate_accuracy_baseline(y_true, baseline_predictions)
    precision = calculate_precision_baseline(y_true, baseline_predictions)
    recall = calculate_recall_baseline(y_true, baseline_predictions)
    confusion = calculate_confusion_matrix_baseline(y_true, baseline_predictions)
    return accuracy, precision, recall, confusion

In [63]:
def baseline(train_data, test_data, target_column):
    class_counts = train_data[target_column].value_counts()
    majority_class = class_counts.idxmax()
    majority_class_count = class_counts.max()
    predictions = [majority_class] * len(test_data)
    return predictions, majority_class, majority_class_count

#print(f'Majority Class: {majority_class}')
#print(f'Majority Class Count: {majority_class_count}')
#print(f'y_true: {y_true}')
#print(f'Baseline Predictions: {baseline_predictions}')

In [70]:
baseline_predictions, majority_class, majority_class_count = baseline(db_treino, db_teste, 'Renda')
y_true = db_teste['Renda'].tolist()

accuracy, precision, recall, confusion = calculate_metrics_baseline()

print(f'Acurácia: {accuracy}')
print(f'Precisão: {precision}')
print(f'Recall: {recall}')
print(f'Matriz de Confusão: {confusion}')

results=[]
results.append([accuracy, precision, recall, confusion])
columns = ['accuracy', 'precision', 'recall', 'confusion']
metricas_res = pd.DataFrame(results, columns=columns)
display(metricas_res)
#display(confusao)

Acurácia: 0.0
Precisão: 0.0
Recall: 0.0
Matriz de Confusão: [[0, 0], [4480, 0]]


,accuracy,precision,recall,confusion
0,0.0,0.0,0.0,"[[0, 0], [4480, 0]]"


# Resultados 3 algoritmos